In [1]:
pip install pyspark==3.3.1

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation_users")\
    .getOrCreate()

23/01/16 15:55:01 WARN Utils: Your hostname, river-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.133.128 instead (on interface ens33)
23/01/16 15:55:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/16 15:55:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/16 15:55:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.json("../../../datalake/twitter_datascience")

In [5]:
df.show()

+--------------------+--------------------+--------------------+------------+
|                data|            includes|                meta|extract_date|
+--------------------+--------------------+--------------------+------------+
|[{737142202481016...|{[{2016-05-30T04:...|{1614960801781252...|  2023-01-16|
|[{156551075009305...|{[{2022-09-02T01:...|{1614960439531819...|  2023-01-16|
|[{161308754706814...|{[{2023-01-11T08:...|{1614960279242276...|  2023-01-16|
|[{109402247573690...|{[{2019-02-08T23:...|{1614960021451976...|  2023-01-16|
|[{101181765595789...|{[{2018-06-27T03:...|{1614959798633795...|  2023-01-16|
|[{137155121135112...|{[{2021-03-15T19:...|{1614959535180898...|  2023-01-16|
|[{142970475005707...|{[{2021-08-23T07:...|{1614959337641762...|  2023-01-16|
|[{18570502, 16149...|{[{2009-01-03T02:...|{1614959152417079...|  2023-01-16|
|[{161308682069857...|{[{2023-01-11T08:...|{1614958997207154...|  2023-01-16|
|[{130630626775391...|{[{2020-09-16T18:...|{1614958862225838...|

In [6]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- impression_count: long (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    

In [7]:
from pyspark.sql import functions as f

In [8]:
df.select(f.explode("includes.users"))

DataFrame[col: struct<created_at:string,id:string,name:string,username:string>]

In [9]:
df.select(f.explode("includes.users").alias("users"))

DataFrame[users: struct<created_at:string,id:string,name:string,username:string>]

In [10]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [11]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [12]:
user_df.show(5)

+--------------------+-------------------+--------------------+--------------+
|          created_at|                 id|                name|      username|
+--------------------+-------------------+--------------------+--------------+
|2016-05-30T04:42:...| 737142202481016832|     Chidambara .ML.|  chidambara09|
|2014-07-04T17:07:...|         2603796666|Annoying Professi...|    StormyCube|
|2009-04-08T14:29:...|           29726352|         UKAuthority|   UKAuthority|
|2012-04-16T07:55:...|          555031989|     Ronald van Loon|Ronald_vanLoon|
|2022-12-29T10:16:...|1608405925886304257|EdTechDigit Innov...|   edtechdigit|
+--------------------+-------------------+--------------------+--------------+
only showing top 5 rows



In [ ]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')